In [1]:
# In case iPython does not find our personalized modules and we want to import them manually
# import sys
# sys.path.append('my/path/to/module/folder')
# import module_of_interest

# We can also make sure what's the main directory iPhython consider for running
# import os
# os.getcwd()

import constants
import requests
import pandas as pd
import numpy as np
import psycopg2
import csv
import datetime
from psycopg2 import sql

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)


In [2]:
def calculate_matchweek(conn):
    """
        Finds out last registered matchweek for current season in Database. 
        Returns last registerered matchweek plus one. If there's any problem 
        with databse or season has not even started, then returns default matchweek "1"
    """
    try:
        cursor = conn.cursor()
        matchweek_query = f"SELECT MAX(\"MatchWeek\") FROM public.match_history WHERE \"Season\" = '{constants.CURRENT_SEASON_TAG}'"
        cursor.execute(matchweek_query)
        matchweek_result = cursor.fetchone()
        if matchweek_result[0] is None:
            return constants.DEFAULT_MATCHWEEK
        else:
            # Next matchweek
            return matchweek_result[0] + 1
    except psycopg2.Error as e:
        print ("An error ocurred in database")
        print (e)
        return constants.DEFAULT_MATCHWEEK
    finally:
        cursor.close()

In [3]:
def upsert_query(to_table, reference_column, dataframe):
    """
        Generates a Composed SQL object that contains all necessary SQL code to 
        perform upserts into a postgresql table from dataframe as source.
        If row attempted to insert is already present in database, then engine
        will skip insertion. 

        Input:
            to_table - target table name in format 'schema.table_name' to generate inserts 
            reference_column - Column name of your table that is considered PK to avoid overwritings
            dataframe - pandas dataframe containing all rows to be inserted
    """
    date_config = sql.SQL('SET datestyle = "ISO, DMY"; ')
    
    insert_header = sql.SQL("INSERT INTO " + to_table + " ({fields})").format(
        fields = sql.SQL(',').join([sql.Identifier(column) for column in dataframe.columns])
    )
    
    values_array = []
    for row in dataframe.iterrows():
        values_array.append(sql.SQL(" ({values})").format(
            values = sql.SQL(',').join([sql.Literal(value) for value in row[1].values])
        ))
    
    insert_values = sql.SQL(' VALUES ') + sql.SQL(',').join(values_array)
    
    insert_condition = sql.SQL(" ON CONFLICT({conflict_col}) DO NOTHING").format(
        conflict_col = sql.Identifier(reference_column)
    )
    
    insert_statement = date_config + insert_header + insert_values + insert_condition
   
    return insert_statement

In [4]:
# -- EXTRACTION

website_frame = pd.read_csv(constants.CSV_SOURCE_URL)

In [19]:
website_frame.head()

,MatchID,Season,MatchWeek,Date,Time,HomeTeam,AwayTeam,FullTimeHomeTeamGoals,FullTimeAwayTeamGoals,FullTimeResult,HalfTimeHomeTeamGoals,HalfTimeAwayTeamGoals,HalfTimeResult,Referee,HomeTeamShots,AwayTeamShots,HomeTeamShotsOnTarget,AwayTeamShotsOnTarget,HomeTeamFouls,AwayTeamFouls,HomeTeamCorners,AwayTeamCorners,HomeTeamYellowCards,AwayTeamYellowCards,HomeTeamRedCards,AwayTeamRedCards,B365HomeTeam,B365Draw,B365AwayTeam,MarketMaxHomeTeam,MarketMaxDraw,MarketMaxAwayTeam,MarketAvgHomeTeam,MarketAvgDraw,MarketAvgAwayTeam,B365Over2.5Goals,B365Under2.5Goals,MarketMaxOver2.5Goals,MarketMaxUnder2.5Goals,MarketAvgOver2.5Goals,MarketAvgUnder2.5Goals,HomeTeamPoints,AwayTeamPoints
0,2023-2024_Burnley_Man City,2023-2024,38,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,A,C Pawson,6,17,1,8,11,8,6,5,0,0,1,0,8.00,5.5,1.33,9.50,5.68,1.39,9.02,5.35,1.35,1.67,2.20,1.71,2.40,1.65,2.27,0,3
1,2023-2024_Arsenal_Nott'm Forest,2023-2024,38,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,H,M Oliver,15,6,7,2,12,12,8,3,2,2,0,0,1.18,7.0,15.00,1.21,8.50,17.50,1.18,7.64,15.67,1.44,2.75,1.45,2.98,1.42,2.85,3,0
2,2023-2024_Bournemouth_West Ham,2023-2024,38,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,D,P Bankes,14,16,5,3,9,14,10,4,1,4,0,0,2.70,3.4,2.55,2.80,3.62,2.75,2.69,3.44,2.64,1.90,2.00,1.95,2.03,1.88,1.94,1,1
3,2023-2024_Brighton_Luton,2023-2024,38,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,H,D Coote,27,9,12,3,11,12,6,7,2,2,0,0,1.33,5.5,9.00,1.36,6.00,10.50,1.33,5.52,9.61,1.62,2.30,1.65,2.45,1.61,2.34,3,0
4,2023-2024_Everton_Fulham,2023-2024,38,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,D,S Attwell,19,9,9,2,12,6,10,4,0,2,0,0,2.20,3.4,3.30,2.30,3.57,3.45,2.24,3.43,3.30,2.01,1.89,2.04,1.92,1.97,1.86,0,3


In [6]:
#

# Remove unwanted columns

unwanted_cols = ["Div", "BWH", "BWD", "BWA", "IWH", "IWD", "IWA", "PSH", "PSD", "PSA", "WHH", "WHD", "WHA", "VCH", "VCD", "VCA",
                 "P>2.5", "P<2.5","AHh", "B365AHH", "B365AHA", "PAHH", "PAHA", "MaxAHH", "MaxAHA", "AvgAHH", "AvgAHA", "B365CH",
                 "B365CD", "B365CA", "BWCH", "BWCD", "BWCA", "IWCH", "IWCD", "IWCA", "PSCH", "PSCD", "PSCA", "WHCH", "WHCD", "WHCA",
                 "VCCH", "VCCD", "VCCA", "MaxCH", "MaxCD", "MaxCA", "AvgCH", "AvgCD", "AvgCA", "B365C>2.5", "B365C<2.5", "PC>2.5",
                 "PC<2.5", "MaxC>2.5", "MaxC<2.5", "AvgC>2.5", "AvgC<2.5", "AHCh", "B365CAHH", "B365CAHA", "PCAHH", "PCAHA", "MaxCAHH",
                 "MaxCAHA", "AvgCAHH", "AvgCAHA"]

website_frame.drop(columns = unwanted_cols, inplace = True)

In [7]:
# Rename columns
website_frame.rename(columns = {"FTHG": "FullTimeHomeTeamGoals",
                               "FTAG": "FullTimeAwayTeamGoals",
                               "FTR": "FullTimeResult",
                               "HTHG": "HalfTimeHomeTeamGoals",
                               "HTAG": "HalfTimeAwayTeamGoals",
                               "HTR": "HalfTimeResult",
                               "HS": "HomeTeamShots",
                               "AS": "AwayTeamShots",
                               "HST": "HomeTeamShotsOnTarget",
                               "AST": "AwayTeamShotsOnTarget",
                               "HF": "HomeTeamFouls",
                               "AF": "AwayTeamFouls",
                               "HC": "HomeTeamCorners",
                               "AC": "AwayTeamCorners",
                               "HY": "HomeTeamYellowCards",
                               "AY": "AwayTeamYellowCards",
                               "HR": "HomeTeamRedCards",
                               "AR": "AwayTeamRedCards",
                               "B365H": "B365HomeTeam",
                               "B365D": "B365Draw",
                               "B365A": "B365AwayTeam",
                               "MaxH": "MarketMaxHomeTeam",
                               "MaxD": "MarketMaxDraw",
                               "MaxA": "MarketMaxAwayTeam",
                               "AvgH": "MarketAvgHomeTeam",
                               "AvgD": "MarketAvgDraw",
                               "AvgA": "MarketAvgAwayTeam",
                               "B365>2.5": "B365Over2.5Goals",
                               "B365<2.5": "B365Under2.5Goals",
                               "Max>2.5": "MarketMaxOver2.5Goals",
                               "Max<2.5": "MarketMaxUnder2.5Goals",
                               "Avg>2.5": "MarketAvgOver2.5Goals",
                               "Avg<2.5": "MarketAvgUnder2.5Goals"},
                   inplace = True)


In [8]:
# Add MatchID column

website_frame.insert(0, "MatchID", constants.CURRENT_SEASON_TAG + "_" + website_frame["HomeTeam"] + "_" + website_frame["AwayTeam"])

In [9]:
# Add season column

website_frame.insert(1, "Season", constants.CURRENT_SEASON_TAG)

In [10]:
# Stablish a connection to Database data source and fetch last game so we can know current matchweek

try:
    connection = psycopg2.connect(
        host = constants.DB_SERVER,
        port = constants.DB_PORT,
        user = constants.DB_USER,
        password = constants.DB_PASSWORD,
        database = constants.DB_NAME
    )
except psycopg2.Error as e:
    print (f'Can not connect to the postgress database "{constants.DB_NAME}". Make sure database server is running')
    print (e)
else:
    print (f'Connection to database "{constants.DB_NAME}" stablished. Listening at port {constants.DB_PORT}')


Connection to database "premier_league" stablished. Listening at port 5432


In [11]:
# Find out current season matchweek
next_matchweek = calculate_matchweek(connection)
next_matchweek

38

In [12]:
# Add MatchWeek column

website_frame.insert(2, "MatchWeek", next_matchweek)

In [13]:
# Add Points columns

conditions = [
     website_frame["FullTimeResult"] == 'H',
     website_frame["FullTimeResult"] == 'D',
     website_frame["FullTimeResult"] == 'A'
]

home_points = [ 3, 1, 0]
away_points = [ 0, 1, 3]

website_frame["HomeTeamPoints"] = np.select(conditions, home_points)
website_frame["AwayTeamPoints"] = np.select(conditions, away_points)

In [14]:
website_frame.head()

,MatchID,Season,MatchWeek,Date,Time,HomeTeam,AwayTeam,FullTimeHomeTeamGoals,FullTimeAwayTeamGoals,FullTimeResult,HalfTimeHomeTeamGoals,HalfTimeAwayTeamGoals,HalfTimeResult,Referee,HomeTeamShots,AwayTeamShots,HomeTeamShotsOnTarget,AwayTeamShotsOnTarget,HomeTeamFouls,AwayTeamFouls,HomeTeamCorners,AwayTeamCorners,HomeTeamYellowCards,AwayTeamYellowCards,HomeTeamRedCards,AwayTeamRedCards,B365HomeTeam,B365Draw,B365AwayTeam,MarketMaxHomeTeam,MarketMaxDraw,MarketMaxAwayTeam,MarketAvgHomeTeam,MarketAvgDraw,MarketAvgAwayTeam,B365Over2.5Goals,B365Under2.5Goals,MarketMaxOver2.5Goals,MarketMaxUnder2.5Goals,MarketAvgOver2.5Goals,MarketAvgUnder2.5Goals,HomeTeamPoints,AwayTeamPoints
0,2023-2024_Burnley_Man City,2023-2024,38,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,A,C Pawson,6,17,1,8,11,8,6,5,0,0,1,0,8.00,5.5,1.33,9.50,5.68,1.39,9.02,5.35,1.35,1.67,2.20,1.71,2.40,1.65,2.27,0,3
1,2023-2024_Arsenal_Nott'm Forest,2023-2024,38,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,H,M Oliver,15,6,7,2,12,12,8,3,2,2,0,0,1.18,7.0,15.00,1.21,8.50,17.50,1.18,7.64,15.67,1.44,2.75,1.45,2.98,1.42,2.85,3,0
2,2023-2024_Bournemouth_West Ham,2023-2024,38,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,D,P Bankes,14,16,5,3,9,14,10,4,1,4,0,0,2.70,3.4,2.55,2.80,3.62,2.75,2.69,3.44,2.64,1.90,2.00,1.95,2.03,1.88,1.94,1,1
3,2023-2024_Brighton_Luton,2023-2024,38,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,H,D Coote,27,9,12,3,11,12,6,7,2,2,0,0,1.33,5.5,9.00,1.36,6.00,10.50,1.33,5.52,9.61,1.62,2.30,1.65,2.45,1.61,2.34,3,0
4,2023-2024_Everton_Fulham,2023-2024,38,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,D,S Attwell,19,9,9,2,12,6,10,4,0,2,0,0,2.20,3.4,3.30,2.30,3.57,3.45,2.24,3.43,3.30,2.01,1.89,2.04,1.92,1.97,1.86,0,3


In [ ]:
# -- LOAD

# Keep up to date postgresql database

insert_statement = upsert_query("public.match_history", "MatchID", website_frame)
print(insert_statement.as_string(connection))

In [18]:
try:
    with connection.cursor() as cursor:
        cursor.execute(insert_statement)
        rowsAffected = cursor.rowcount
        connection.commit()
except psycopg2.Error as e:
    print (f'Can not connect to the postgress database "{constants.DB_NAME}". Make sure database server is running')
    print (e)
else:
    print (f"New Rows in Match History: {rowsAffected}")
finally:
    cursor.close()

New Rows in Match History: 0


In [ ]:
# Keep up to date master dataset


In [ ]:
# Keep up to date Kaggle dataset